# 40. Leaning neural networks

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

np.random.seed(17)
tf.random.set_seed(17)

In [2]:
df = pd.read_csv('prepared_posts_df_40.csv')
df.head()

,nsfw,my,images,videos,text_len,rating,tsin,tcos,in_comm,Длиннопост,...,Своими руками,My Little Pony,Музыка,Мошенничество,Отношения,Стихи,Зима,СССР,Еда,other
0,0,1,2,0,135,683,-0.705620,0.708590,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,113,480,-0.705414,0.708795,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,301,-0.704950,0.709257,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,4,0,0,425,-0.704537,0.709667,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,726,617,-0.704021,0.710179,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df.drop(columns=['rating']).to_numpy()
Y = df['rating'].to_numpy()


In [6]:
val_frac = 0.15
test_frac = 0.15

x_train = X[:round((1 - val_frac - test_frac) * len(X))]
x_valid = X[round((1 - val_frac - test_frac) * len(X)):round((1 - test_frac) * len(X))]
x_test = X[round((1 - test_frac) * len(X)):]

y_train = Y[:round((1 - val_frac - test_frac) * len(Y))]
y_valid = Y[round((1 - val_frac - test_frac) * len(Y)):round((1 - test_frac) * len(Y))]
y_test = Y[round((1 - test_frac) * len(Y)):]
print(f'x_train={x_train.shape}, x_valid={x_valid.shape}, x_test={x_test.shape}')
print(f'y_train={y_train.shape}, y_valid={y_valid.shape}, y_test={y_test.shape}')      

x_train=(183804, 59), x_valid=(39386, 59), x_test=(39387, 59)
y_train=(183804,), y_valid=(39386,), y_test=(39387,)


In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.linear)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               6000      
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 26,301
Trainable params: 26,301
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.save_weights('homework_init_weights.h5')

In [9]:
model.compile(
    loss=tf.keras.losses.MSE,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.MeanAbsoluteError(),
             tf.keras.metrics.RootMeanSquaredError()]
)

In [10]:
model.evaluate(X, Y)

8206/8206 [==============================] - 7s 844us/step - loss: 356148.8438 - mean_absolute_error: 444.9341 - root_mean_squared_error: 596.7820


[356148.84375, 444.9341125488281, 596.7820434570312]

In [12]:
%%time
history = model.fit(X, Y, batch_size=5000, epochs=300)

Epoch 1/300
53/53 [==============================] - 1s 13ms/step - loss: 124505.3359 - mean_absolute_error: 174.6613 - root_mean_squared_error: 352.8531
Epoch 2/300
53/53 [==============================] - 1s 13ms/step - loss: 117418.3438 - mean_absolute_error: 167.3121 - root_mean_squared_error: 342.6636
Epoch 3/300
53/53 [==============================] - 1s 13ms/step - loss: 115695.5312 - mean_absolute_error: 164.8993 - root_mean_squared_error: 340.1405
Epoch 4/300
53/53 [==============================] - 1s 13ms/step - loss: 115447.5547 - mean_absolute_error: 164.4283 - root_mean_squared_error: 339.7758
Epoch 5/300
53/53 [==============================] - 1s 13ms/step - loss: 113466.9688 - mean_absolute_error: 161.6831 - root_mean_squared_error: 336.8486
Epoch 6/300
53/53 [==============================] - 1s 12ms/step - loss: 112394.8203 - mean_absolute_error: 160.2639 - root_mean_squared_error: 335.2534
Epoch 7/300
53/53 [==============================] - 1s 13ms/step - loss: 11

In [ ]:
%%time
history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), batch_size=50000, epochs=100)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
%%time
model.compile(
    loss=tf.keras.losses.MSE,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=[tf.keras.metrics.MeanAbsoluteError(),
             tf.keras.metrics.RootMeanSquaredError()]
)

history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), batch_size=50000, epochs=100)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
%%time
history3 = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), batch_size=50000, epochs=500)

In [ ]:
plt.plot(history3.history['loss'])
plt.plot(history3.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
%%time
model.compile(
    loss=tf.keras.losses.MSE,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    metrics=[tf.keras.metrics.MeanAbsoluteError(),
             tf.keras.metrics.RootMeanSquaredError()]
)

history4 = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), batch_size=50000, epochs=500)

plt.plot(history4.history['loss'])
plt.plot(history4.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
model.evaluate(x_test, y_test)